In [42]:
#Make necessary imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

#Loading Data in
data = pd.read_csv('../input/ks-projects-201801.csv')

**Looking at basic datatypes and unique values of each column**

In [43]:
def load_and_explore(data):
    print("############# PREVIEW ########################")
    print(data.head())
    print("############# DATA TYPES ########################")
    print(data.info())
    print("############# NO. OF UNIQUE VALS ########################")
    print(data.nunique())

load_and_explore(data)

############# PREVIEW ########################
           ID      ...      usd_goal_real
0  1000002330      ...            1533.95
1  1000003930      ...           30000.00
2  1000004038      ...           45000.00
3  1000007540      ...            5000.00
4  1000011046      ...           19500.00

[5 rows x 15 columns]
############# DATA TYPES ########################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
ID                  378661 non-null int64
name                378657 non-null object
category            378661 non-null object
main_category       378661 non-null object
currency            378661 non-null object
deadline            378661 non-null object
goal                378661 non-null float64
launched            378661 non-null object
pledged             378661 non-null float64
state               378661 non-null object
backers             378661 non-null int64
country             378661 non-null object
u

This shows us that the launched and deadline columns, which should be datetypes are columns and so should be converted. The state column should be converted to a boolean value, and needs to be dealt with as there should only be two states. Let's deal with this now.

In [44]:
print(data["state"].value_counts())
def convert_to_bool(item):
    if item=="failed":
        return 0
    elif item=="successful":
        return 1
    else:
        return 2

data["state"] = data["state"].apply(convert_to_bool)
print(data["state"].value_counts())

failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64
0    197719
1    133956
2     46986
Name: state, dtype: int64


We see that 46986 columns are neither successful or failed, and are cancelced, still live, suspended or undefined. So we delete these. Now, set the id column as id, and start generating some new features. First, we will look at the date. From the launched and deadline columns, the natural features to consider are:
1. Duration
2. Month it was launched in
3. Month of Deadline
4. Quarter it was launched in
5. Quarter of Deadline

The year doesn't make much sense to consider as we will be making predictions on live projects and so the year doesn't make sense. 

In [45]:
#Set ID
data = data.set_index('ID')
data.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
ID,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,0,0,GB,0.0,0.0,1533.95
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,0,15,US,100.0,2421.0,30000.00
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,0,3,US,220.0,220.0,45000.00
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,0,1,US,1.0,1.0,5000.00
1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,2,14,US,1283.0,1283.0,19500.00


In [46]:
#Create new features from launched, deadline

#First, convert to datetime
data["launched"] = pd.to_datetime(data["launched"], format="%Y-%m-%d %H:%M:%S")
data["deadline"] = pd.to_datetime(data["deadline"], format="%Y-%m-%d")

#Create new features

#Duration in days
data["duration"] = (data["deadline"] - data["launched"]).dt.days

#Quarter, month of launched and deadline date
data["launch_month"] = data["launched"].dt.month
data["launch_quarter"] = data["launched"].dt.quarter
data["deadline_month"] = data["deadline"].dt.month
data["deadline_quarter"] = data["deadline"].dt.quarter

#The launch hour may also have an impact, as it may affect when/if it goes viral
data["launch_hour"] = data["launched"].dt.hour
data[['launched','deadline', 'duration','launch_month', 'launch_quarter', 'deadline_month', 'deadline_quarter']].head()

,launched,deadline,duration,launch_month,launch_quarter,deadline_month,deadline_quarter
ID,,,,,,,
1000002330,2015-08-11 12:12:28,2015-10-09,58,8,3,10,4
1000003930,2017-09-02 04:43:57,2017-11-01,59,9,3,11,4
1000004038,2013-01-12 00:20:50,2013-02-26,44,1,1,2,1
1000007540,2012-03-17 03:24:11,2012-04-16,29,3,1,4,2
1000011046,2015-07-04 08:35:03,2015-08-29,55,7,3,8,3


Let's create new features from the title now. Performing sentiment analysis or other NLP techniques doesn't make much sense as each title is quite different as it describes the specific project, and may have an impact on making the title vectorization the same as the category. Therefore, I will stick to basic title vectorization techniques.

In [47]:
#Create new features from title

#Length of title
data["title_length"] = data["name"].apply(lambda x: len(str(x)))

#Number of words
data["title_words"] = data["name"].apply(lambda x: len(str(x).split(' ')))

#Number of symbols
data['title_symbols'] = data["name"].apply(lambda x: str(x).count('!') + str(x).count('?'))

data[['title_length', 'title_words', 'title_symbols']].head()

,title_length,title_words,title_symbols
ID,,,
1000002330,31,6,0
1000003930,45,8,0
1000004038,14,3,1
1000007540,49,7,0
1000011046,58,8,0


Now finally, some additional features will be created based on the backers/pledged. Make any inferences from the pledged column is risky because of the fact that we don't know when the data was scraped, so I am not sure if considering that column heavily is a smart idea. I will only look at one metric: amount pledged per backer.

In [48]:
#Create new pledged feature

#New column for if there are any backers
data["backers_exist"] = np.where(data["backers"]>0, "True", "False")

#Make a mask for rows that contain backers
mask_backers_exist = (data["backers"]>0)

#Enter 0 for where backers don't exist, and the pledged per backer for where they do
data['pledged_per_backer'] = 0
data.loc[mask_backers_exist, 'pledged_per_backer'] = data["pledged"] / data["backers"]

print(data[["pledged", "backers", "pledged_per_backer"]].head())
print("MAX VALS")
print(data[["pledged", "backers","pledged_per_backer"]].mean())

            pledged  backers  pledged_per_backer
ID                                              
1000002330      0.0        0            0.000000
1000003930   2421.0       15          161.400000
1000004038    220.0        3           73.333333
1000007540      1.0        1            1.000000
1000011046   1283.0       14           91.642857
MAX VALS
pledged               9682.979339
backers                105.617476
pledged_per_backer      72.318140
dtype: float64


Now we must find those columns with NaN values, and deal with them accordingly. 

In [49]:
#Find columns with NaN values
data.isna().any()[lambda x:x]
data.isna().sum()

name                     4
category                 0
main_category            0
currency                 0
deadline                 0
goal                     0
launched                 0
pledged                  0
state                    0
backers                  0
country                  0
usd pledged           3797
usd_pledged_real         0
usd_goal_real            0
duration                 0
launch_month             0
launch_quarter           0
deadline_month           0
deadline_quarter         0
launch_hour              0
title_length             0
title_words              0
title_symbols            0
backers_exist            0
pledged_per_backer       0
dtype: int64

We see that the name, usd pledged columns have NaN values. Therefore, we must deal with this on a case by case basis. Let's look at the name column's null values.

In [50]:
data[data["name"].isnull()]

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,duration,launch_month,launch_quarter,deadline_month,deadline_quarter,launch_hour,title_length,title_words,title_symbols,backers_exist,pledged_per_backer
ID,,,,,,,,,,,,,,,,,,,,,,,,,
1848699072,NaN,Narrative Film,Film & Video,USD,2012-02-29,200000.0,2012-01-01 12:35:31,100.0,0,1,US,100.00,100.00,200000.00,58,1,1,2,1,12,3,1,0,True,100.000000
634871725,NaN,Video Games,Games,GBP,2013-01-06,2000.0,2012-12-19 23:57:48,196.0,0,12,GB,317.73,316.05,3224.97,17,12,4,1,1,23,3,1,0,True,16.333333
648853978,NaN,Product Design,Design,USD,2016-07-18,2500.0,2016-06-18 05:01:47,0.0,2,0,US,0.00,0.00,2500.00,29,6,2,7,3,5,3,1,0,False,0.000000
796533179,NaN,Painting,Art,USD,2011-12-05,35000.0,2011-11-06 23:55:55,220.0,0,5,US,220.00,220.00,35000.00,28,11,4,12,4,23,3,1,0,True,44.000000


As we can see, since there are only 4, we can replace the features created from the title for these with the means. Let's look at the USD pledged column.

In [51]:
data[data["usd pledged"].isnull()]

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,duration,launch_month,launch_quarter,deadline_month,deadline_quarter,launch_hour,title_length,title_words,title_symbols,backers_exist,pledged_per_backer
ID,,,,,,,,,,,,,,,,,,,,,,,,,
1000694855,STREETFIGHTERZ WHEELIE MURICA,Film & Video,Film & Video,USD,2014-09-20,6500.0,2014-08-06 21:28:36,555.00,2,0,"N,0""",NaN,555.00,6500.00,44,8,3,9,3,21,29,3,0,False,0.0
100149523,Duncan Woods - Chameleon EP,Music,Music,AUD,2015-08-25,4500.0,2015-08-04 12:05:17,4767.00,2,0,"N,0""",NaN,3402.08,3211.53,20,8,3,8,3,12,27,5,0,False,0.0
1003023003,The Making of Ashley Kelley's Debut Album,Music,Music,USD,2015-04-09,3500.0,2015-03-10 20:06:13,3576.00,2,0,"N,0""",NaN,3576.00,3500.00,29,3,1,4,2,20,41,7,0,False,0.0
1003130892,Butter Side Down Debut Album,Music,Music,USD,2015-11-26,6000.0,2015-11-02 22:09:19,7007.80,2,0,"N,0""",NaN,7007.80,6000.00,23,11,4,11,4,22,28,5,0,False,0.0
1003629045,Chase Goehring debut EP,Music,Music,USD,2016-03-21,3000.0,2016-02-23 03:09:49,3660.38,2,0,"N,0""",NaN,3660.38,3000.00,26,2,1,3,1,3,23,4,0,False,0.0
1004013077,Spencer Capier Instrumental Project 2015,Music,Music,CAD,2014-10-10,4000.0,2014-09-10 22:51:01,4760.00,2,0,"N,0""",NaN,4251.14,3572.39,29,9,3,10,4,22,40,5,0,False,0.0
1004126342,LUKAS LIGETI'S 50TH BIRTHDAY FESTIVAL: ORIGINA...,Music,Music,USD,2015-06-11,5000.0,2015-05-15 02:12:35,6370.00,2,0,"N,0""",NaN,6370.00,5000.00,26,5,2,6,2,2,58,8,1,False,0.0
1004255433,The Battle For Breukelen: A Neighborhood Epic,Film & Video,Film & Video,USD,2015-11-07,6000.0,2015-10-10 18:45:24,6695.00,2,0,"N,0""",NaN,6695.00,6000.00,27,10,4,11,4,18,45,7,0,False,0.0
1004298993,"""Tamboura Plays Violin"" - a collection of Pop ...",Music,Music,USD,2015-03-28,2750.0,2015-02-11 19:28:27,2926.00,2,0,"N,0""",NaN,2926.00,2750.00,44,2,1,3,1,19,58,10,1,False,0.0


From here, we see that the columns that are null for usd pledged have a weird country value as well. We do know the currency, and so getting the country shouldn't be difficult. From this, we can replace the country column and the usd pledged column by making an exchange rate dictionary. So, let's do this. First, we find all the currencies. Then, find the country from the currency, and then the usd pledged from the currency.

In [52]:
#Get Currencies and Countries
print(data["currency"].value_counts())
print(data["country"].value_counts())

USD    295365
GBP     34132
EUR     17405
CAD     14962
AUD      7950
SEK      1788
MXN      1752
NZD      1475
DKK      1129
CHF       768
NOK       722
HKD       618
SGD       555
JPY        40
Name: currency, dtype: int64
US      292627
GB       33672
CA       14756
AU        7839
DE        4171
N,0"      3797
FR        2939
IT        2878
NL        2868
ES        2276
SE        1757
MX        1752
NZ        1447
DK        1113
IE         811
CH         761
NO         708
HK         618
BE         617
AT         597
SG         555
LU          62
JP          40
Name: country, dtype: int64


In [53]:
#Make dictionary mapping currency to country code
curr_to_country  = {
    "USD": ["US"],
    "GBP": ["GB"],
    "EUR": ["DE", "FR", "IT", "NL","ES", "IE", "BE", "AT", "LU"], #Denmark, France, Italy, Netherlands, Spain, Ireland, Belgium, Austria, Luxembourg
    "CAD": ["CA"],
    "AUD": ["AU"],
    "SEK": ["SE"],
    "MXN": ["MX"],
    "NZD": ["NZ"],
    "DKK": ["DK"],
    "CHF": ["CH"],
    "NOK": ["NO"],
    "HKD": ["HK"],
    "SGD": ["SG"],
    "JPY": ["JP"]
}

#Find all rows with bad country names
mask_bad_countries = (data["country"]=='N,0"')

#Randomly get country from the EUR array, because we can not know which country it was
data["good_country"] = data["currency"].apply(lambda x: random.choice(curr_to_country[x]))

#Replace the bad countries with the fixed country. Don't do for all as that will lose truth due to the EUR
data.loc[mask_bad_countries, "country"] = data["good_country"]

data["country"].value_counts()

US    295365
GB     34132
CA     14962
AU      7950
DE      4192
FR      2962
IT      2895
NL      2892
ES      2297
SE      1788
MX      1752
NZ      1475
DK      1129
IE       826
CH       768
NO       722
BE       640
AT       623
HK       618
SG       555
LU        78
JP        40
Name: country, dtype: int64

We can see that the faulty data has been removed

In [54]:
#Now find USD pledged based on currency
curr_usd_exchange_rate = {
    "USD": 1.0,
    "CAD": 0.75,
    "MXN": 0.052,
    "SGD": 0.73,
    "EUR": 1.12,
    "AUD": 0.69,
    "CHF": 1.00,
    "DKK": 0.15,
    "GBP": 1.26,
    "HKD": 0.13,
    "JPY": 0.0092,
    "NOK": 0.11,
    "NZD": 0.65,
    "SEK": 0.11
}

#Get exchange rate
data["exchange_rate"] = data["currency"].apply(lambda x: curr_usd_exchange_rate[x])

#Get the good value of usd pledged
data["usd_pledged_new"] = data["pledged"] * data["exchange_rate"]

#Replace all bad values with the good value
data.loc[mask_bad_countries, "usd pledged"] = data["usd_pledged_new"]
data.isna().sum()

name                  4
category              0
main_category         0
currency              0
deadline              0
goal                  0
launched              0
pledged               0
state                 0
backers               0
country               0
usd pledged           0
usd_pledged_real      0
usd_goal_real         0
duration              0
launch_month          0
launch_quarter        0
deadline_month        0
deadline_quarter      0
launch_hour           0
title_length          0
title_words           0
title_symbols         0
backers_exist         0
pledged_per_backer    0
good_country          0
exchange_rate         0
usd_pledged_new       0
dtype: int64

Now, all the NaN values for the country and the usd pledged have been fixed! Now, we can just drop the NaN rows with the names, as they are only 4 rows like this.

In [56]:
data = data.dropna()
data.isna().sum()

name                  0
category              0
main_category         0
currency              0
deadline              0
goal                  0
launched              0
pledged               0
state                 0
backers               0
country               0
usd pledged           0
usd_pledged_real      0
usd_goal_real         0
duration              0
launch_month          0
launch_quarter        0
deadline_month        0
deadline_quarter      0
launch_hour           0
title_length          0
title_words           0
title_symbols         0
backers_exist         0
pledged_per_backer    0
good_country          0
exchange_rate         0
usd_pledged_new       0
dtype: int64

Now that all of the NaN and faulty values have been dealt with, we can start filtering the important features.To start, we will do some manual analysis here. 